First, we start by predicting using the maskrcnn model.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import torch
from PIL import Image
import transforms as T


In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

path = 'x/'

In [ ]:
path = 'x/'
outputPath = 'outputVal/maskrcnn/'
os.makedirs(outputPath,exist_ok=True)
transforms = get_transform(train=False)
for imName in list(sorted(os.listdir(path))):
    
    model = torch.jit.load("instanceSegmentation.pth")
    model = model.cpu()
    model.eval()
    #img, _ = dataset_test[i]
    img = Image.open(path+imName).convert("RGB")
    # Aqui hay que transformar la imagen con los transforms. 
    img,_ = transforms(img,img)
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # put the model in evaluation mode
    model.to(device)
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])
        
    im = prediction[1][0]['masks'][0, 0].mul(255).byte().cpu().numpy()
    for j in range(1,len(prediction[1][0]['masks'])): 
        im = np.maximum(im,prediction[1][0]['masks'][j, 0].mul(255).byte().cpu().numpy())
    
    ima = Image.fromarray(im)
    # Guardar con el nombre adecuado. 
    ima.save(outputPath+imName)
    

Now, we make the predictions using the HRNet model that obtains both the cytoplasm and the nucleus. 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# HRNet

model = torch.jit.load("hrnet.pth")
model = model.cpu()
model.eval()

In [ ]:
import torchvision.transforms as transforms
def transform_image(image):
    my_transforms = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image_aux = image
    return my_transforms(image_aux).unsqueeze(0).to(device)

In [ ]:
from fastai.basics import *
from fastai.vision import models
from fastai.vision.all import *
from fastai.metrics import *
from fastai.data.all import *
from fastai.callback import *

from pathlib import Path

inputpath = Path(path)

In [ ]:
outputPath = 'outputVal/hrnet/'
os.makedirs(outputPath,exist_ok=True)
for file in inputpath.ls():
    image = Image.open(file)
    h,w=image.shape
    image = transforms.Resize((1000,1333))(image)
    tensor = transform_image(image=image)
    model.to(device)
    with torch.no_grad():
        outputs = model(tensor)
    outputs = torch.argmax(outputs,1)
    mask = np.array(outputs.cpu())
    mask[mask==1]=20
    mask[mask==2]=40
    mask=np.reshape(mask,(1000*1333))
    new  = np.zeros((1000*1333,3),dtype='uint8')

    for i,v in enumerate(mask):
        new[i]=[v,v,v]


    maskRGBShow = Image.fromarray(np.reshape(new,(1000,1333,3)))
    maskRGBShow.save(outputPath+file.name)
    

Now, we make the predictions using the Deeplab model that obtains both the cytoplasm and the nucleus. 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# HRNet

model = torch.jit.load("deeplab.pth")
model = model.cpu()
model.eval()

In [ ]:
outputPath = 'outputVal/deeplab/'
os.makedirs(outputPath,exist_ok=True)
for file in inputpath.ls():
    image = Image.open(file)
    h,w=image.shape
    image = transforms.Resize((1000,1333))(image)
    tensor = transform_image(image=image)
    model.to(device)
    with torch.no_grad():
        outputs = model(tensor)
    outputs = torch.argmax(outputs,1)
    mask = np.array(outputs.cpu())
    mask[mask==1]=20
    mask[mask==2]=40
    mask=np.reshape(mask,(1000*1333))
    new  = np.zeros((1000*1333,3),dtype='uint8')

    for i,v in enumerate(mask):
        new[i]=[v,v,v]


    maskRGBShow = Image.fromarray(np.reshape(new,(1000,1333,3)))
    maskRGBShow.save(outputPath+file.name)
    

Finally, we combine the predictions of the hrnet and deeplab models.

In [ ]:
import cv2

In [ ]:
maskrcnnPath = 'outputVal/maskrcnn/'
hrnetPath = 'outputVal/hrnet/'
deeplabPath = 'outputVal/deeplab/'

In [ ]:
outputModelsPath = 'outputVal/combination/'
os.makedirs(outputModelsPath,exist_ok=True)

for imName in list(sorted(os.listdir(path))):
    im1 = cv2.imread(maskrcnnPath+imName)
    (h,w,d) = im1.shape
    im2 = cv2.imread(hrnetPath+imName)
    im2 = cv2.resize(im2,(w,h))
    
    im3 = cv2.imread(deeplabPath+imName)
    im3 = cv2.resize(im3,(w,h))
    
    im2 = np.maximum(im2,im3)
    
    cv2.imwrite(outputModelsPath+imName,im2)
    
    
    